In [2]:
import torch
from timm import create_model

In [5]:
model = create_model('deit_tiny_patch16_224', pretrained=True, num_classes=1000)

In [6]:
model.load_state_dict(torch.load("C:/Users/seonahryu/Desktop/brp1/deit_tiny_patch16_224-a1311bcf.pth"), strict=False)

C:\Users\seonahryu\AppData\Local\Temp\ipykernel_86428\3522241460.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("C:/Users/seonahryu/Dow

_IncompatibleKeys(missing_keys=['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias', 'blocks.2.ml

In [7]:
model.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()


In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch.nn as nn

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # DeiT 모델 입력 크기에 맞춤
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = ImageFolder(root="C:/Users/seonahryu/Desktop/brp1/ILSVRC2012_img_val", transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [10]:
import time
from thop import profile # FLOPs

# 모델 FLOPs 계산 함수
def compute_flops(model, input_size=(1, 3, 224, 224)):
    flops, params = profile(model, inputs=(torch.randn(input_size),))
    return flops

# 모델 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    loss_fn = nn.CrossEntropyLoss()

    start_time = time.time()  # 평가 시작 시간 기록

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = loss_fn(outputs, labels)

    end_time = time.time()  # 평가 종료 시간 기록
    elapsed_time = end_time - start_time  # 총 소요 시간 계산

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.3f}%')
    print(f'Loss: {loss.item():.3f}')
    
    # 이미지 처리 속도 계산
    im_per_sec = total / elapsed_time
    print(f'Processing speed: {im_per_sec:.2f} images/sec')
    
    # FLOPs 계산
    flops = compute_flops(model)
    print(f'FLOPs: {flops:.2f}')

In [15]:
evaluate_model(model, test_loader)

Accuracy: 69.956%
Loss: 2.464
Processing speed: 26.88 images/sec
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
FLOPs: 1078633728.00
